In [2]:
from datasets import load_dataset
dataset = load_dataset("webdataset", data_dir="ai-lectures-spring-24", streaming=True)

/home/emilia/.cache/pypoetry/virtualenvs/video-rag-model-_HJFAOPk-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
print(dataset)

IterableDatasetDict({
    train: IterableDataset({
        features: ['mp4', 'info.json', 'en.vtt', 'json', '__key__', '__url__'],
        num_shards: 1
    })
})


In [4]:
stream = dataset["train"]
print(stream)

IterableDataset({
    features: ['mp4', 'info.json', 'en.vtt', 'json', '__key__', '__url__'],
    num_shards: 1
})


In [ ]:
for sample in stream:
    print(sample)

In [1]:
from IPython.display import Video

Video("ai-lectures-spring-24/videos/9CGGh6ivg68/9CGGh6ivg68.mp4")

In [9]:
import re

def getTimestamps(vtt_path):
    '''Takes in a .en.vtt file path and returns a list of timestamped words'''
    word_list = []

    with open(vtt_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    
    first_word_time_pattern = re.compile(r'(\d{2}:\d{2}:\d{2}.\d{3}) -->')
    timestamp_pattern = re.compile(r'<(\d{2}:\d{2}:\d{2}.\d{3})><c>([^<]*)</c>')
    first_word_pattern = re.compile(r'^([^<]*)<')

    first_word_time = None
    for line in lines:
        first_word_time_match = first_word_time_pattern.findall(line)
        if len(first_word_time_match) != 0:
            first_word_time = first_word_time_match[0]

        timestamp_matches = timestamp_pattern.findall(line)
        if len(timestamp_matches) != 0:
            first_word = first_word_pattern.findall(line)[0]

            word_list.append({"start": first_word_time, "word": first_word.strip()})
            for time_str, word in timestamp_matches:
                word_list.append({"start": time_str, "word": word.strip()})


    return word_list

In [10]:
text = getTimestamps('ai-lectures-spring-24/videos/9CGGh6ivg68/9CGGh6ivg68.en.vtt')

In [11]:
for x in text:
    print(f'{x['start']} {x['word']}')

00:00:00.719 in
00:00:00.919 this
00:00:01.120 video
00:00:01.880 I
00:00:01.959 would
00:00:02.159 like
00:00:02.320 to
00:00:02.480 start
00:00:02.760 the
00:00:03.000 discussion
00:00:03.560 about
00:00:03.959 convolutional
00:00:04.600 new
00:00:04.880 networks
00:00:05.319 which
00:00:05.400 is
00:00:05.600 another
00:00:06.399 architecture
00:00:07.399 of
00:00:07.799 uh
00:00:08.000 neural
00:00:08.280 networks
00:00:08.679 that
00:00:08.880 we
00:00:09.400 are
00:00:09.639 going
00:00:09.840 to
00:00:10.000 see
00:00:10.400 specifically
00:00:10.960 kind
00:00:11.160 of
00:00:11.799 engineered
00:00:12.799 uh
00:00:12.960 to
00:00:13.719 um
00:00:14.200 address
00:00:14.879 problems
00:00:15.360 that
00:00:15.559 we
00:00:15.679 are
00:00:15.879 facing
00:00:16.240 in
00:00:16.440 computer
00:00:16.840 vision
00:00:17.160 I
00:00:17.240 want
00:00:17.400 to
00:00:17.520 start
00:00:17.880 this
00:00:18.359 discussion
00:00:19.359 with
00:00:19.920 um
00:00:20.119 just
00:00:20.

In [26]:
import re

def parseAndCleanVtt(vtt_path):
    word_list = []

    with open(vtt_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    
    first_word_time_pattern = re.compile(r'(\d{2}:\d{2}:\d{2}.\d{3}) -->')
    timestamp_pattern = re.compile(r'<(\d{2}:\d{2}:\d{2}.\d{3})><c>([^<]*)</c>')
    first_word_pattern = re.compile(r'^([^<]*)<')

    words_to_remove = ['um','uh']

    first_word_time = None
    prev_word = ''
    for line in lines:
        first_word_time_match = first_word_time_pattern.findall(line)
        if len(first_word_time_match) != 0:
            first_word_time = first_word_time_match[0]

        timestamp_matches = timestamp_pattern.findall(line)
        if len(timestamp_matches) != 0:
            first_word = first_word_pattern.findall(line)[0]

            first_word = first_word.strip()

            if first_word not in words_to_remove and first_word != prev_word:
                word_list.append({"start": first_word_time, "word": first_word})
                prev_word = first_word

            for time_str, word in timestamp_matches:
                word = word.strip()
                if word not in words_to_remove and word != prev_word:
                    word_list.append({"start": time_str, "word": word})
                    prev_word = word


    return word_list

In [27]:
cleaned_text = parseAndCleanVtt('ai-lectures-spring-24/videos/9CGGh6ivg68/9CGGh6ivg68.en.vtt')

In [29]:
with open('cleaned_timestamps.txt', 'w') as file:
    for timestamped_word in cleaned_text:
        file.write(f'{timestamped_word['start']} {timestamped_word['word']}\n')

In [28]:
with open('cleaned_transcript.txt', 'w') as file:
    for timestamped_word in cleaned_text:
        file.write(f"{timestamped_word['word']} ")

In [17]:
for x in text:
    print(x['word'], end=' ')

in this video I would like to start the discussion about convolutional new networks which is another architecture of uh neural networks that we are going to see specifically kind of engineered uh to um address problems that we are facing in computer vision I want to start this discussion with um just showing you a picture and uh if I ask you uh to tell me what would actually be the first object that you pay attention to then most people will probably respond to with yellow cab and it's not really accident that lot of cabs in a lot of capitals are painted uh yellow U and of course that attention uh to bright colors originates from thousands of years of evolution where people are of course trained to pay attention to uh bright colors which are typically sources of food such as fruits or in some instances sources of uh danger such as the yellow tiger that is coming towards us uh so kind of joking aside I think you can um associate uh this kind of image with the following kind of processin

In [12]:
import json

def load_json_from_file(file_path):
    with open(file_path, 'r') as file:
        # Load the JSON data from the file
        data = json.load(file)
    return data

In [15]:
import requests
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt

def show_storyboard_images(file_path):
    json_data = load_json_from_file(file_path)

    # Iterate through the formats to fetch the images
    for format_info in json_data['formats']:
        print(f"Displaying storyboard images for format: {format_info['format']}")
        
        # Iterate through the fragments to get image URLs
        for fragment in format_info['fragments']:
            image_url = fragment['url']
            try:
                # Fetch the image
                response = requests.get(image_url)
                img = Image.open(BytesIO(response.content))
                
                # Display the image
                plt.imshow(img)
                plt.axis('off')  # Hide axes
                plt.show()
                
            except Exception as e:
                print(f"Failed to load image from {image_url}. Error: {e}")

In [ ]:
show_storyboard_images('ai-lectures-spring-24/videos/9CGGh6ivg68/9CGGh6ivg68.info.json')